## Download model from HuggingFace

In [9]:
import mlflow
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from mlflow.models import infer_signature

# Define Model Name and MLflow Tracking URI
HF_MODEL_NAME = 'HuggingFaceTB/SmolLM2-135M-Instruct'  # Model from Hugging Face
MLFLOW_TRACKING_URI = 'http://localhost:5000'  # Adjust if MLflow is running elsewhere

# Set MLflow Tracking Server
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Load Model and Tokenizer from Hugging Face
model = AutoModelForCausalLM.from_pretrained(HF_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)

# Save Model Locally Before Logging to MLflow
MODEL_DIR = "SmolLM2-135M-Instruct"
model.save_pretrained(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

('SmolLM2-135M-Instruct/tokenizer_config.json',
 'SmolLM2-135M-Instruct/special_tokens_map.json',
 'SmolLM2-135M-Instruct/vocab.json',
 'SmolLM2-135M-Instruct/merges.txt',
 'SmolLM2-135M-Instruct/added_tokens.json',
 'SmolLM2-135M-Instruct/tokenizer.json')

## Save model onto MLflow

In [10]:
import os

os.environ["MLFLOW_HTTP_REQUEST_MAX_SIZE"] = str(256 * 1024 * 1024)  # 256MB
os.environ["MLFLOW_UPLOAD_BUFFER_SIZE"] = str(128 * 1024 * 1024)  # 128MB
os.environ["MLFLOW_MAX_ARTIFACT_SIZE"] = str(1 * 1024 * 1024 * 1024)  # 1GB
os.environ["MLFLOW_HTTP_REQUEST_TIMEOUT"] = "600"  # 10 minutes
os.environ["MLFLOW_TCP_KEEPALIVE"] = "1"

In [12]:
# Define Experiment Name (Matching Hugging Face Model Name)
EXPERIMENT_NAME = 'HuggingFaceTB'
RUN_NAME = 'SmolLM2-135M-Instruct'

# Create (or Get) Experiment in MLflow
mlflow.set_experiment(EXPERIMENT_NAME)


with mlflow.start_run(run_name=RUN_NAME) as run:
    # Log Model Directory as Artifacts
    mlflow.log_artifacts(MODEL_DIR)

    print(f"✅ Model '{HF_MODEL_NAME}' successfully saved and logged as artifacts in MLflow!")
    print(f"🧪 Experiment: {EXPERIMENT_NAME}")
    print(f"📌 Run Name: {RUN_NAME}")
    print(f"🔗 Run ID: {run.info.run_id}")

2025/01/22 00:48:29 INFO mlflow.tracking.fluent: Experiment with name 'HuggingFaceTB' does not exist. Creating a new experiment.


✅ Model 'HuggingFaceTB/SmolLM2-135M-Instruct' successfully saved and logged as artifacts in MLflow!
🧪 Experiment: HuggingFaceTB
📌 Run Name: SmolLM2-135M-Instruct
🔗 Run ID: ea8a642cc39d4d70a72376c2a44f9108
🏃 View run SmolLM2-135M-Instruct at: http://localhost:5000/#/experiments/3/runs/ea8a642cc39d4d70a72376c2a44f9108
🧪 View experiment at: http://localhost:5000/#/experiments/3


In [13]:
!rm -rf SmolLM2-135M-Instruct/

## Download model from MLflow (verification that everything is working fine)

In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define MLflow Run & Artifact Path
EXPERIMENT_NAME = 'HuggingFaceTB'
DEST_PATH = 'SmolLM2-135M-Instruct'

client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
runs = client.search_runs(experiment.experiment_id, order_by=["start_time desc"])

if not runs:
    raise ValueError("No runs found for experiment 'SmolLM2-360M-Instruct'")

latest_run_id = runs[0].info.run_id
print(f"🔗 Latest Run ID: {latest_run_id}")

# List artifacts in this run
artifacts = client.list_artifacts(latest_run_id)
for artifact in artifacts:
    print(f"📂 Artifact: {artifact.path}")

🔗 Latest Run ID: ea8a642cc39d4d70a72376c2a44f9108
📂 Artifact: config.json
📂 Artifact: generation_config.json
📂 Artifact: merges.txt
📂 Artifact: model.safetensors
📂 Artifact: special_tokens_map.json
📂 Artifact: tokenizer.json
📂 Artifact: tokenizer_config.json
📂 Artifact: vocab.json


In [22]:
MODEL_DOWNLOAD_PATH = mlflow.artifacts.download_artifacts(run_id=latest_run_id, dst_path=DEST_PATH)
print(f"✅ Model downloaded to: {MODEL_DOWNLOAD_PATH}")

✅ Model downloaded to: /root/madrigal/src/SmolLM2-135M-Instruct/


## Load and run model

In [37]:
# Load Model & Tokenizer
model = AutoModelForCausalLM.from_pretrained(MODEL_DOWNLOAD_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DOWNLOAD_PATH)

input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Encode Input Without System Instructions
inputs = tokenizer.encode(input_text, return_tensors="pt")

# Generate Response
outputs = model.generate(inputs, temperature=0.2, top_p=0.9, do_sample=True)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("🤖 Model Response:", response)

🤖 Model Response: system
You are a helpful AI assistant named SmolLM, trained by Hugging Face
user
What is gravity?
assistant
Gravity is a fundamental force of nature that attracts objects with mass towards each other. It is a


In [38]:
!rm -rf SmolLM2-135M-Instruct

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
